In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns 

import string 
import nltk
from nltk.corpus import stopwords 
from wordcloud import WordCloud 
nltk.download('stopwords')

import tensorflow as tf 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from sklearn.model_selection import train_test_split 
from keras.callbacks import EarlyStopping, ReduceLROnPlateau 

import warnings 
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/kaggle/input/newemails/Ham.csv
/kaggle/input/newemails/Spam.csv
/kaggle/input/email-spam-classification-dataset-csv/emails.csv
/kaggle/input/emails/Ham_csv.numbers
/kaggle/input/emails/Spam_csv.numbers


# What are all these imports??
* seaborn - Python data visualization library based on Matplotlib
* string - provides string operations 
* nltk - Natural Language Toolkit library 
    * stopwords - list of common stopwords in multiple languages
* WordCloud - word cloud visualizations or visual representation of word data 
* pad sequences - pads sequences to the same length 

In [23]:
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix 

# Create a DataFrame 

In [40]:
ham = pd.read_csv('/kaggle/input/sodone/Ham 2.csv')
spam = pd.read_csv('/kaggle/input/sodone/Spam 2.csv')

1 = spam, 0 = not spam

In [48]:
ham

,ID,Thread,Date,From,To,Subject,Snippet,Labels,Link
0,190502420645d278,190502420645d278,"Tue, 25 Jun 2024 16:04:15 -0000","""AnitaB.org"" <newsletter@anitab.org>",nh104@wellesley.edu,🎉 GHC 24 Registration Is Now OPEN!,Join your community at GHC 24! View in Browser...,"CATEGORY_PROMOTIONS, UNREAD, IMPORTANT, INBOX",https://mail.google.com/mail/u/0/#inbox/190502...
1,1904f6f9899fe290,1904f6f9899fe290,"Tue, 25 Jun 2024 12:47:02 +0000","""Doug Dyer from @TheFundCFO Newsletter"" <thefu...",nh104@wellesley.edu,"#162 VCs Closing Q2 w/ Playbook, Model, Valuat...","👋 Hi, I&#39;m Doug! Welcome to @TheFundCFO cre...","IMPORTANT, CATEGORY_UPDATES",https://mail.google.com/mail/u/0/#inbox/1904f6...
2,1904f401a79c62cf,1904f401a79c62cf,"Tue, 25 Jun 2024 11:54:40 +0000","""Zvi Mowshowitz from Don't Worry About the Vas...",nh104@wellesley.edu,Monthly Roundup #19: June 2024,"Looks like we made it. Yes, the non-AI world s...","IMPORTANT, CATEGORY_UPDATES, INBOX",https://mail.google.com/mail/u/0/#inbox/1904f4...
3,1904e3f0b57de850,1904e3f0b57de850,"Tue, 25 Jun 2024 07:14:32 +0000",Handshake <handshake@notifications.joinhandsha...,nh104@wellesley.edu,"2 new jobs match your saved search: ""Venture C...",Handshake 2 new jobs match your search for &qu...,"UNREAD, IMPORTANT, CATEGORY_UPDATES",https://mail.google.com/mail/u/0/#inbox/1904e3...
4,1904c49ed39265c1,1904c49ed39265c1,"Mon, 24 Jun 2024 22:07:10 +0000",Marisa Hettinger via Handshake <handshake@noti...,nh104@wellesley.edu,Take the Lead: Follow PMG Today & Launch Your ...,Marisa from PMG Digital Marketing just message...,"IMPORTANT, CATEGORY_UPDATES, INBOX",https://mail.google.com/mail/u/0/#inbox/1904c4...
...,...,...,...,...,...,...,...,...,...
109,18fbba8376067a14,18fbba8376067a14,"Mon, 27 May 2024 16:05:00 -0400",Yutong Wu <yw108@wellesley.edu>,"CS Students <cs-students@wellesley.edu>, MAS-s...",Recruiting Participants for MIT Media Lab Stud...,Hello Everyone! I hope you&#39;re making the m...,"IMPORTANT, CATEGORY_FORUMS",https://mail.google.com/mail/u/0/#inbox/18fbba...
110,18fbaa690a5db832,18fbaa690a5db832,"Mon, 27 May 2024 17:22:09 +0200 (CET)","""AI for Good Global Summit, ITU"" <ai@itu.int>",Natalie Ho <nh104@wellesley.edu>,[AI for Good Summit] Happening this week! Impo...,Important Information for Participants Dear co...,"UNREAD, IMPORTANT, CATEGORY_PERSONAL",https://mail.google.com/mail/u/0/#inbox/18fbaa...
111,18fb910214cc2312,18fb910214cc2312,"Mon, 27 May 2024 07:59:44 +0000",OxfordSpeaks <info@oxford-speaks.com>,<nh104@wellesley.edu>,Upcoming Events,Brenda Hale and Gideon Levy View this email in...,"IMPORTANT, CATEGORY_UPDATES",https://mail.google.com/mail/u/0/#inbox/18fb91...
112,18fb755840b95489,18fb755840b95489,"Sun, 26 May 2024 18:55:50 -0500",VAISU <info@vaisu.ai>,undisclosed-recipients:;,VAISU 2024: Many Thanks!,"Hi Everyone, We wanted to extend our heartfelt...","UNREAD, IMPORTANT, CATEGORY_PERSONAL",https://mail.google.com/mail/u/0/#inbox/18fb75...


In [47]:
spam

,ID,Thread,Date,From,To,Subject,Snippet,Labels,Link
0,190500a8244c8f00,190500a8244c8f00,"Tue, 25 Jun 2024 09:26:38 -0600",FlixBus <noreply@email.flixbus.com>,<natalieho@gmail.com>,Celebrate July 4th with an Epic Getaway! 🇺🇸🚌,‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ...,"CATEGORY_PROMOTIONS, UNREAD, SPAM",https://mail.google.com/mail/u/0/#inbox/190500...
1,1905004dec205714,1905004dec205714,"Tue, 25 Jun 2024 10:30:13 -0500",Clifford Sundberg <djdjgdjd455@gmail.com>,natalie.ho@gmail.com,Thank you for your order!4336998208,Hey there Great to see you again! Your account...,"UNREAD, CATEGORY_UPDATES, SPAM",https://mail.google.com/mail/u/0/#inbox/190500...
2,1904fbd33edae16a,1904fbd33edae16a,"Tue, 25 Jun 2024 10:11:57 -0400 (EDT)",Redneck Wine Company <contact@redneckwinecompa...,natalieho@gmail.com,Hartford Court's Russian River Chardonnay - Va...,Vote For Us in the TBT&#39;s Best of the Best ...,"CATEGORY_PROMOTIONS, UNREAD, SPAM",https://mail.google.com/mail/u/0/#inbox/1904fb...
3,1904fa29daa4b30b,1904fa29daa4b30b,"Tue, 25 Jun 2024 07:33:00 -0600",Kipling <kipling-usa@mail.kipling-usa.com>,<natalie.ho@gmail.com>,Just Launched! Our Minions Collection!,Go Ba-na-nas!!! Hurry before it sells out! Vie...,"CATEGORY_PROMOTIONS, UNREAD, SPAM",https://mail.google.com/mail/u/0/#inbox/1904fa...
4,1904f61c3e923595,1904f61c3e923595,"Tue, 25 Jun 2024 12:31:43 +0000",JetBlue <jetblueairways@email.jetblue.com>,natalieho@gmail.com,Do tell the kids: 4 destinations to book right...,"Orlando, San Juan, The Palm Beaches, Turks &am...","UNREAD, CATEGORY_UPDATES, SPAM",https://mail.google.com/mail/u/0/#inbox/1904f6...
...,...,...,...,...,...,...,...,...,...
554,18fb500f7e7976de,18fb500f7e7976de,"Sun, 26 May 2024 13:03:48 +0000",The Enterprise-Journal <publisher@enterprise-j...,<natalieho@gmail.com>,Weekly Most Read Local Posts from The Enterpri...,Weekly Most Read The most popular local storie...,"UNREAD, CATEGORY_UPDATES, SPAM",https://mail.google.com/mail/u/0/#inbox/18fb50...
555,18fb496163bea48a,18fb496163bea48a,"Sun, 26 May 2024 13:08:05 +0200",SHEIN <shein@edm.shein.com>,natalieho@gmail.com,🎁Αγοράστε 3 Πάρτε 1 Δώρο,Θα τηρηθεί σειρά προτεραιότητας. ‌ ‌ ‌ ‌ ‌ ‌ ‌...,"CATEGORY_PROMOTIONS, UNREAD, SPAM",https://mail.google.com/mail/u/0/#inbox/18fb49...
556,18fb491999be702d,18fb491999be702d,"Sun, 26 May 2024 11:03:00 +0000",The Enterprise-Journal <publisher@enterprise-j...,<natalieho@gmail.com>,Weekly Most Read Comic Panels from The Enterpr...,Weekly Comics The most popular comic panels of...,"CATEGORY_PROMOTIONS, UNREAD, SPAM",https://mail.google.com/mail/u/0/#inbox/18fb49...
557,18fb49190e732109,18fb49190e732109,"Sun, 26 May 2024 11:02:53 +0000",The Enterprise-Journal <publisher@enterprise-j...,<natalieho@gmail.com>,Weekly Most Read Political Cartoons from The E...,Weekly Cartoons The most popular political car...,"CATEGORY_PROMOTIONS, UNREAD, SPAM",https://mail.google.com/mail/u/0/#inbox/18fb49...


In [41]:
ham
spam

,ID,Thread,Date,From,To,Subject,Snippet,Labels,Link
0,190500a8244c8f00,190500a8244c8f00,"Tue, 25 Jun 2024 09:26:38 -0600",FlixBus <noreply@email.flixbus.com>,<natalieho@gmail.com>,Celebrate July 4th with an Epic Getaway! 🇺🇸🚌,‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ...,"CATEGORY_PROMOTIONS, UNREAD, SPAM",https://mail.google.com/mail/u/0/#inbox/190500...
1,1905004dec205714,1905004dec205714,"Tue, 25 Jun 2024 10:30:13 -0500",Clifford Sundberg <djdjgdjd455@gmail.com>,natalie.ho@gmail.com,Thank you for your order!4336998208,Hey there Great to see you again! Your account...,"UNREAD, CATEGORY_UPDATES, SPAM",https://mail.google.com/mail/u/0/#inbox/190500...
2,1904fbd33edae16a,1904fbd33edae16a,"Tue, 25 Jun 2024 10:11:57 -0400 (EDT)",Redneck Wine Company <contact@redneckwinecompa...,natalieho@gmail.com,Hartford Court's Russian River Chardonnay - Va...,Vote For Us in the TBT&#39;s Best of the Best ...,"CATEGORY_PROMOTIONS, UNREAD, SPAM",https://mail.google.com/mail/u/0/#inbox/1904fb...
3,1904fa29daa4b30b,1904fa29daa4b30b,"Tue, 25 Jun 2024 07:33:00 -0600",Kipling <kipling-usa@mail.kipling-usa.com>,<natalie.ho@gmail.com>,Just Launched! Our Minions Collection!,Go Ba-na-nas!!! Hurry before it sells out! Vie...,"CATEGORY_PROMOTIONS, UNREAD, SPAM",https://mail.google.com/mail/u/0/#inbox/1904fa...
4,1904f61c3e923595,1904f61c3e923595,"Tue, 25 Jun 2024 12:31:43 +0000",JetBlue <jetblueairways@email.jetblue.com>,natalieho@gmail.com,Do tell the kids: 4 destinations to book right...,"Orlando, San Juan, The Palm Beaches, Turks &am...","UNREAD, CATEGORY_UPDATES, SPAM",https://mail.google.com/mail/u/0/#inbox/1904f6...
...,...,...,...,...,...,...,...,...,...
554,18fb500f7e7976de,18fb500f7e7976de,"Sun, 26 May 2024 13:03:48 +0000",The Enterprise-Journal <publisher@enterprise-j...,<natalieho@gmail.com>,Weekly Most Read Local Posts from The Enterpri...,Weekly Most Read The most popular local storie...,"UNREAD, CATEGORY_UPDATES, SPAM",https://mail.google.com/mail/u/0/#inbox/18fb50...
555,18fb496163bea48a,18fb496163bea48a,"Sun, 26 May 2024 13:08:05 +0200",SHEIN <shein@edm.shein.com>,natalieho@gmail.com,🎁Αγοράστε 3 Πάρτε 1 Δώρο,Θα τηρηθεί σειρά προτεραιότητας. ‌ ‌ ‌ ‌ ‌ ‌ ‌...,"CATEGORY_PROMOTIONS, UNREAD, SPAM",https://mail.google.com/mail/u/0/#inbox/18fb49...
556,18fb491999be702d,18fb491999be702d,"Sun, 26 May 2024 11:03:00 +0000",The Enterprise-Journal <publisher@enterprise-j...,<natalieho@gmail.com>,Weekly Most Read Comic Panels from The Enterpr...,Weekly Comics The most popular comic panels of...,"CATEGORY_PROMOTIONS, UNREAD, SPAM",https://mail.google.com/mail/u/0/#inbox/18fb49...
557,18fb49190e732109,18fb49190e732109,"Sun, 26 May 2024 11:02:53 +0000",The Enterprise-Journal <publisher@enterprise-j...,<natalieho@gmail.com>,Weekly Most Read Political Cartoons from The E...,Weekly Cartoons The most popular political car...,"CATEGORY_PROMOTIONS, UNREAD, SPAM",https://mail.google.com/mail/u/0/#inbox/18fb49...


In [43]:
print(ham.keys())

Index(['ID', 'Thread', 'Date', 'From', 'To', 'Subject', 'Snippet', 'Labels',
       'Link'],
      dtype='object')


In [46]:
ham_essential = ham[["From", "Subject", "Snippet"]]
spam_essential = spam[["From", "Subject", "Snippet"]]

ham_essential
spam_essential

,From,Subject,Snippet
0,FlixBus <noreply@email.flixbus.com>,Celebrate July 4th with an Epic Getaway! 🇺🇸🚌,‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ...
1,Clifford Sundberg <djdjgdjd455@gmail.com>,Thank you for your order!4336998208,Hey there Great to see you again! Your account...
2,Redneck Wine Company <contact@redneckwinecompa...,Hartford Court's Russian River Chardonnay - Va...,Vote For Us in the TBT&#39;s Best of the Best ...
3,Kipling <kipling-usa@mail.kipling-usa.com>,Just Launched! Our Minions Collection!,Go Ba-na-nas!!! Hurry before it sells out! Vie...
4,JetBlue <jetblueairways@email.jetblue.com>,Do tell the kids: 4 destinations to book right...,"Orlando, San Juan, The Palm Beaches, Turks &am..."
...,...,...,...
554,The Enterprise-Journal <publisher@enterprise-j...,Weekly Most Read Local Posts from The Enterpri...,Weekly Most Read The most popular local storie...
555,SHEIN <shein@edm.shein.com>,🎁Αγοράστε 3 Πάρτε 1 Δώρο,Θα τηρηθεί σειρά προτεραιότητας. ‌ ‌ ‌ ‌ ‌ ‌ ‌...
556,The Enterprise-Journal <publisher@enterprise-j...,Weekly Most Read Comic Panels from The Enterpr...,Weekly Comics The most popular comic panels of...
557,The Enterprise-Journal <publisher@enterprise-j...,Weekly Most Read Political Cartoons from The E...,Weekly Cartoons The most popular political car...


# DataFrame 
* a dataframe is 2D data structure with columns 
* describe() is a method that returns the stats of a dataframe including its count, mean, standard deviation etc 

In [20]:
df.describe(include='object')

,Email No.
count,5172
unique,5172
top,Email 1
freq,1


In [9]:
df.shape

(115, 1)

In [24]:
df.columns()
df.target()

TypeError: 'Index' object is not callable

In [26]:
df['input'] = 'TEXT1: ' + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor 

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U16'), dtype('int64')) -> None

In [22]:
df.input.head()

0    0
1    0
2    0
3    0
4    0
Name: input, dtype: int64

# Tokenization 

Datasets is a library by HuggingFace that allows easily accessing and sharing datasets for Audio, Computer Vision, and NLP tasks. 

# Pandas Data Frame vs HF Dataset 
* Pandas DataFrame - 2D labeled data structure w columns of potentially different types 
* HF Dataset - library and data structure to store and access datasets 

In [ ]:
from datasets import Dataset, DatasetDict

#converts a pandas DataFrame into a Hugging Face Dataset object 
ds = Dataset.from_pandas(df)

In [ ]:
ds

This is a model developed by Microsoft called DeBERTa. It's the third version and has fewer parameters than the 'base' or 'large versions. 

In [ ]:
model_nm = 'microsoft/deberta-v3-small'

# Data Preprocessing 

In [11]:
# Remove stopwords 
def remove_stopwords(text):
    stop_words = stopwords.words('english')
    star_words = []
    
    #store important words
    for word in str(text).split():
        word = word.lower()
        if word not in stop_words:
            star_words.append(word)
    output = " ".join(star_words)
    return output


In [57]:
remove_stopwords(ham_essential)


'0 "anitab.org" <newsletter@anitab.org> 1 "doug... subject 0 🎉 ghc 24 registration open! 1 #162 vcs closi... snippet 0 join community ghc 24! view browser... 1 👋 h... dtype: object'

In [56]:
ham_essential = ham_essential.apply(lambda text: remove_stopwords(text))
ham_essential.head()

From       0 "anitab.org" <newsletter@anitab.org> 1 "doug...
Subject    0 🎉 ghc 24 registration open! 1 #162 vcs closi...
Snippet    0 join community ghc 24! view browser... 1 👋 h...
dtype: object

In [58]:
spam_essential = spam_essential.apply(lambda text: remove_stopwords(text))
spam_essential.head()

From       0 flixbus <noreply@email.flixbus.com> 1 cliffo...
Subject    0 celebrate july 4th epic getaway! 🇺🇸🚌 1 thank...
Snippet    0 ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ...
dtype: object

# World2Vec Conversion 

In [60]:
train_X, test_X, train_Y, test_Y = train_test_split(ham_essential, spam_essential, test_size = 0.2, random_state = 42)

In [62]:
train_X.head()

Subject    0 🎉 ghc 24 registration open! 1 #162 vcs closi...
Snippet    0 join community ghc 24! view browser... 1 👋 h...
dtype: object

# Transformers and AutoTokenizer 

* transformers - HF library with wide range of pre-trained models for NLP 
* AutoModelForSequenceClassification - loads appropriate model architecture 
* AutoTokenizer - loads appropriate tokenizer (processes text so that model can understand) 
* tokenizer splits the input text into tokens --> numbers --> adds any special tokens 

In [63]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_X)

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer 
tokz = AutoTokenizer.from_pretrained(model_nm)

In [ ]:
tokz.tokenize("Hi my name is Natalie.")

In [ ]:
tokz('Hello my name is Nat')

In [ ]:
def tok_func(x): return tokz(x['input'])

Batch mapping allows you to speed up processing and freely control the size of the generated dataset. HF's Tokenizers library works faster with batches because it parallelizes the tokenization of all the examples in a batch. 

In [ ]:
tok_ds = ds.map(tok_func, batched=True)

In [ ]:
row = 